In [1]:
from bertopic import BERTopic
import pandas as pd
from tqdm import tqdm

path = ''

2023-04-12 10:55:49.646078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 10:55:49.748569: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-12 10:55:49.748592: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-12 10:55:50.286962: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
topic_model = BERTopic.load("{}topic_model_24-04".format(path))

In [3]:
df = pd.read_parquet('{}pdf-phrases_24-04.parquet'.format(path))
df

,doi,file_name,text,phrases
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Arti...","[Quim., Nova, Vol., 33, No. 2, 321-323, 2010 A..."
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,..."
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from ���������...,[Biological activity of astilbin from ��������...
...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R....","[Short Reports 1671 Dichrographe Mark III, R..."
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,...","[Phytochemrstry, Vol. 23, No 6, pp., 1X33-129..."
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. p...,"[Journal of Natural Products Vol. 47., No. 2...."
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679..."


In [4]:
topics = []
for phrases_list in tqdm(df.phrases.values):
    phrases_topics = []
    for phrase in phrases_list:
        phrases_topics.append(topic_model.find_topics(phrase)[0])
    topics.append(list(set([item for sublist in phrases_topics for item in sublist])))

 66%|██████▌   | 256/390 [08:44<05:38,  2.52s/it]

In [5]:
from copy import deepcopy

topics_flat = pd.Series([item for sublist in topics for item in sublist])
topics_count = {}
for topic in topics_flat.unique():
    topics_count[topic] = len(topics_flat[topics_flat == topic])/len(topics_flat.unique())
to_remove = [-1, 0]
for key, value in topics_count.items():
    if key == -1 or key == 0:
        continue
    if value > 0.8:
        to_remove.append(key)
print("to remove: {}".format(len(to_remove)))
topics_filtered = deepcopy(topics)
for index, topic_list in enumerate(topics_filtered):
    new_list = []
    for topic in topic_list:
        if topic not in to_remove:
            new_list.append(topic)
    topics_filtered[index] = new_list
print("topics flat original: {}".format(len(topics_flat)))
print("topics flat filtered: {}".format(len([item for sublist in topics_filtered for item in sublist])))


to remove: 2
topics flat original: 93150
topics flat filtered: 92897


In [6]:
df['topics'] = topics_filtered
df

,doi,file_name,text,phrases,topics
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...,"[1, 2, 3, 4, 5, 7, 9, 11, 13, 14, 16, 17, 18, ..."
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Arti...","[Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Art...","[1, 2, 514, 4, 517, 518, 7, 5, 9, 521, 13, 525..."
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,...","[512, 1, 2, 3, 4, 517, 6, 518, 8, 9, 10, 7, 51..."
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...,"[512, 1, 2, 514, 4, 517, 518, 7, 6, 9, 5, 520,..."
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from ���������...,"[Biological activity of astilbin from, �������...","[514, 662, 6, 390, 520, 393, 778, 143, 787, 53..."
...,...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R....","[Short Reports 1671 Dichrographe Mark III, R...","[1, 2, 514, 513, 517, 5, 7, 3, 521, 522, 518, ..."
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,...","[Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296...","[1, 2, 517, 6, 7, 518, 521, 9, 11, 519, 13, 12..."
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. p...,[Journal of Natural Products Vol. 47. No. 2. ...,"[2, 514, 6, 7, 518, 530, 18, 19, 23, 537, 538,..."
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679...","[1, 2, 514, 5, 6, 7, 8, 521, 9, 11, 518, 13, 5..."


In [7]:
import numpy as np
len_list = [len(topic_list) for topic_list in df.topics.values]
df.iloc[len_list.index(np.max(len_list))]

doi                              10.1016/S0031-9422(00)89351-6
file_name                    10.1016@S0031-9422(00)89351-6.pdf
text         Phytochemwro.  1978, Vol 17, pp 517-521 Pergam...
phrases      [Phytochemwro.  1978, Vol 17, pp 517-521, Perg...
topics       [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...
Name: 344, dtype: object

In [8]:
df.to_parquet('{}topics03-05.parquet'.format(path))